In [ ]:
!pip install -U -q "google-generativeai>=0.7.2"

In [ ]:
import time
import google.generativeai as genai
from IPython.display import Markdown, HTML, display

import cv2
import matplotlib.pyplot as plt
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=api_key)

#### Search

In [ ]:
for model in genai.list_models():
  if '002' in model.name: print(model.name)

models/gemini-1.5-pro-002
models/gemini-1.5-flash-002
models/imagen-3.0-generate-002


In [ ]:
''' Search grounding '''
model = genai.GenerativeModel('gemini-1.5-flash-002')
response = model.generate_content('SF tech week very brief')
display(response.text)

'SF Tech Week is a large-scale technology event held annually in San Francisco, featuring conferences, workshops, networking events, and more, showcasing the latest advancements and trends in the tech industry.  It attracts a global audience of professionals, investors, and enthusiasts.\n'

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash-002', tools='google_search_retrieval')
response = model.generate_content('When and where is SF tech week')
resp = response.candidates[0]
print(resp.content.parts[0])

text: "Based on the provided text, San Francisco Tech Week in 2024 took place from October 7th to October 13th.  For 2025, it is scheduled for October 6th to October 12th.  \n\nSeveral sources mention specific events within the larger Tech Week framework,  occurring at various locations across San Francisco.  The events themselves are organized by a number of venture capitalists, entrepreneurs, and community leaders in the Bay Area, not by a single entity.  The SOMA district is suggested as a good place to stay due to its proximity to many events.  The nature of events varies, including fireside chats, panels, and workshops.\n"



In [ ]:
Markdown(resp.content.parts[0].text)

Based on the provided text, San Francisco Tech Week in 2024 took place from October 7th to October 13th.  For 2025, it is scheduled for October 6th to October 12th.  

Several sources mention specific events within the larger Tech Week framework,  occurring at various locations across San Francisco.  The events themselves are organized by a number of venture capitalists, entrepreneurs, and community leaders in the Bay Area, not by a single entity.  The SOMA district is suggested as a good place to stay due to its proximity to many events.  The nature of events varies, including fireside chats, panels, and workshops.


In [ ]:
chunks = resp.grounding_metadata.grounding_chunks
for chunk in chunks: print(chunk)

web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrxJdyfnSr9w62lqgRcVAehx5Xhfb6CVvPh_Yr25kGLbIHuRadDGR6nLO81krpAF6bG7wNJ8oaNT6MLCgFuhB0258_mtikJ2bjkPDJtfcm-pKSixwdMqMORw76E01iaOemq6Xk5-Isx7L0If8mCw1_79hxxFN1xHYRP8epGH3Q=="
  title: "grayarea.org"
}

web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrzRkMAR8GGZ84KFxWbbYNXV8VZIBmvrEyUPBs4Z4j0dK8r2lkhgRnaMQT-eRxn0gU3COgZiZFUm_Sj3-VL8c1SMDb_iRUNVG422XxWbF5ApCgRIAn8w"
  title: "tech-week.com"
}

web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrwGx-DoAdNyrUbae8swBh6a3ps7WgXFfXcfD8a-yf-3XEt075fTFjyA-Kdd4cShs8vmDdgCn8c6tnA1iEJUVkd-56OdPOit2rheTXmT4EAMkagP1w=="
  title: "sf-techweek.com"
}



In [ ]:
HTML(resp.grounding_metadata.search_entry_point.rendered_content)

In [ ]:
supports = resp.grounding_metadata.grounding_supports
for support in supports:
  print(support)

segment {
  end_index: 104
  text: "Based on the provided text, San Francisco Tech Week in 2024 took place from October 7th to October 13th."
}
grounding_chunk_indices: 0
confidence_scores: 0.897299

segment {
  start_index: 106
  end_index: 164
  text: "For 2025, it is scheduled for October 6th to October 12th."
}
grounding_chunk_indices: 1
confidence_scores: 0.775663

segment {
  start_index: 453
  end_index: 544
  text: "The SOMA district is suggested as a good place to stay due to its proximity to many events."
}
grounding_chunk_indices: 2
confidence_scores: 0.902794838

segment {
  start_index: 546
  end_index: 623
  text: "The nature of events varies, including fireside chats, panels, and workshops."
}
grounding_chunk_indices: 2
confidence_scores: 0.989756167



In [ ]:
import io
markdown_buffer = io.StringIO()
markdown_buffer.write('Supported text: \n\n')
for support in supports:
  markdown_buffer.write(' * ')
  markdown_buffer.write(
      response.text[support.segment.start_index : support.segment.end_index])

  for i in support.grounding_chunk_indices:
    chunk = chunks[i].web
    markdown_buffer.write(f'<sup>[{i+1}]</sup>')

  markdown_buffer.write('\n\n')

''' footnotes/citations '''
markdown_buffer.write('Citations: \n\n')
for i, chunk in enumerate(chunks, start=1):
  markdown_buffer.write(
      f'* {i}: [{chunk.web.title}]({chunk.web.uri})\n')

Markdown(markdown_buffer.getvalue())

Supported text: 

 * SF Tech Week is a large-scale technology event held annually in San Francisco, featuring conferences, wo<sup>[1]</sup>

 * shops, networking events, and more, showcasing the latest <sup>[2]</sup>

 * <sup>[3]</sup>

 * <sup>[3]</sup>

Citations: 

* 1: [grayarea.org](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrxJdyfnSr9w62lqgRcVAehx5Xhfb6CVvPh_Yr25kGLbIHuRadDGR6nLO81krpAF6bG7wNJ8oaNT6MLCgFuhB0258_mtikJ2bjkPDJtfcm-pKSixwdMqMORw76E01iaOemq6Xk5-Isx7L0If8mCw1_79hxxFN1xHYRP8epGH3Q==)
* 2: [tech-week.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrzRkMAR8GGZ84KFxWbbYNXV8VZIBmvrEyUPBs4Z4j0dK8r2lkhgRnaMQT-eRxn0gU3COgZiZFUm_Sj3-VL8c1SMDb_iRUNVG422XxWbF5ApCgRIAn8w)
* 3: [sf-techweek.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrwGx-DoAdNyrUbae8swBh6a3ps7WgXFfXcfD8a-yf-3XEt075fTFjyA-Kdd4cShs8vmDdgCn8c6tnA1iEJUVkd-56OdPOit2rheTXmT4EAMkagP1w==)


#### Dynamic grounding

In [ ]:
nosearch_model = genai.GenerativeModel('gemini-1.5-flash-002')
chat = nosearch_model.start_chat()

In [ ]:
resp = chat.send_message('Hello!')
resp = chat.send_message(
    'When and where is SF tech week', tools = 'google_search_retrieval')
Markdown(resp.text)

Based on the provided sources, San Francisco Tech Week in 2024 took place from October 7th to October 13th.  Source [1] indicates that in 2025, it will be held from October 6th to 12th.  Note that this information refers to a series of events under the "Tech Week" umbrella, not a single, centrally organized event.  Many different companies and organizations host individual events during this period.


In [ ]:
resp = chat.send_message('Hello!')
resp = chat.send_message(
    'When and where is SF tech wee', tools = 'google_search_retrieval')
Markdown(resp.text)

There isn't one single "SF Tech Week."  Instead, several events use the "Tech Week" branding or occur during periods often referred to as Tech Week.  Based on your provided sources, here's a breakdown of tech-related events in San Francisco in 2025:


* **October 6-12, 2025:**  A series of events across San Francisco are grouped under the "Tech Week" banner.  These are individually hosted by different companies, not a single large-scale conference. The location varies depending on the specific event.

* **January 10-19, 2025:** SF HealthTech Week takes place across San Francisco and the Bay Area, coinciding with the JP Morgan Healthcare Conference.  Again, this encompasses multiple events at various locations.

* **June 16-20, 2025:** Human Tech Week focuses on the intersection of human wellness, AI, and future potential.  This is a dedicated event series with specific locations to be determined closer to the event dates.


To find precise locations and details for any of these events, you'll need to consult the individual event websites (links provided in the original sources).  Each event will have its own schedule and location information.


In [ ]:
HTML(resp.candidates[0].grounding_metadata.search_entry_point.rendered_content)

In [ ]:
search_config = {
  'dynamic_retrieval_config': {'mode': 'MODE_DYNAMIC', 'dynamic_threshold': 0.5}}

search_model = genai.GenerativeModel(
  'gemini-1.5-flash-002', tools={'google_search_retrieval': search_config})

chat = search_model.start_chat()
response = chat.send_message('Hello friendly chatbot')
resp = response.candidates[0]
score = resp.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f'first turn: {score=}')

response = chat.send_message('When and where is SF tech wee')
resp = response.candidates[0]
score = resp.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f'second turn: {score=}')

display(Markdown(response.text))

first turn: score=0.25767046213150024
second turn: score=0.711559534072876


Based on the provided sources, SF Tech Week 2024 took place from October 7th to 13th,  and SF Tech Week 2025 is scheduled for October 6th to 12th.  The events are held in San Francisco, primarily in the SOMA district.


In [ ]:
HTML(resp.grounding_metadata.search_entry_point.rendered_content)